In [14]:
from cgeniepy.foram import ForamModel as fm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.transforms as mtransforms

mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['font.family'] = 'Fira Sans'

In [16]:
mod_paths = [
    "../model/muffin.CB.GIteiiva.BASESFeTDTL_rb_foramecogem2.1",
    "../model/worjh2.RpCO2_Rp13CO2.Albani.0ka.historical",
    "../model/worjh2.RpCO2_Rp13CO2.Albani.2100.4deg",
]

fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(10, 8),
                        subplot_kw={'projection': ccrs.Robinson()})
trans = mtransforms.ScaledTranslation(10/72, -5/72, fig.dpi_scale_trans)

def foram_presence(model_path, foram_type):
    model = fm(model_path)
    data = model.select_foramtype(foram_type).biomass().isel(time=-1)
    data.where(data.array>1E-9, ifso=1, otherwise=0, overwrite_array=True)
    return data

def f_richness(model_path):
    fr = foram_presence(model_path, "bn") + foram_presence(model_path, "bs") + \
        foram_presence(model_path, "sn") + foram_presence(model_path, "ss")
    return  fr

labels = ['a', 'b', 'c']
titles = ["LGM", "Modern (2020s)", r"Future, 2100 (+4$^\circ$C)"]

for i, ipath in enumerate(mod_paths):
    ax = axs.flat[i]
    ax.text(0, 0.95, labels[i], transform=ax.transAxes + trans, weight="bold", fontsize=12)
    ax.set_title(titles[i])
    f_richness(ipath).plot_map(ax=ax, colorbar=False)

cax = fig.add_axes([0.40, 0.07, 0.2, 0.02])
cb = mpl.colorbar.ColorbarBase(cax, orientation='horizontal', cmap='viridis',
                               norm=mpl.colors.Normalize(0, 4), ticks=[0, 1, 2, 3, 4],
                               label='Functional richness')

In [17]:
fig.savefig("../output/functional_richness.png", dpi=400, bbox_inches='tight')